In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read in training data
train_data = pd.read_csv("lab1_train.data", sep="\t", header=None)
train_data_x = train_data[train_data.columns[:-1]]
train_data_y = train_data[train_data.columns[-1]]
train_data.shape

(1109, 4909)

In [3]:
# Read in test data
test_data = pd.read_csv("lab1_test.data", sep="\t", header=None)
test_data_x = test_data[test_data.columns[:-1]]
test_data_y = test_data[test_data.columns[-1]]
test_data.shape

(1110, 4909)

In [4]:
# First check with scikit learn version
from sklearn.naive_bayes import BernoulliNB
mod0 = BernoulliNB()
mod0.fit(train_data_x, train_data_y)

BernoulliNB()

In [5]:
# Performance on training data
sum(mod0.predict(train_data_x) == train_data_y)

912

In [6]:
# Performance on test data
sum(mod0.predict(test_data_x) == test_data_y)

880

In [7]:
from typing import Mapping
from collections import defaultdict
from functools import partial


def calc_log_prob(series: pd.Series, alpha=1.0) -> Mapping[int, float]:
    counts = series.value_counts() + alpha / 2
    probs = np.log(counts / (counts.sum() + alpha))
    return defaultdict(lambda: np.log(1e-6), probs)

# Here, define your own Naive Bayesian model
class MyNaiveBayes():
    def __init__(self, alpha=0):
        # Here you might want to initialize e.g. counters for n_y and n_iy needed to estimate P(y) and P(x_i|y)
        self.alpha = alpha
        self.prob_y = None
        self.prob_xi_y = None

    def fit(self, data_x: pd.DataFrame, data_y: pd.Series):
        # Here you need to go through the pandas DataFrame objects data_x and data_y and update the counters.
        # You may also here (or alternatively in predict) estimate P(y) and P(x_i|y).
        # First, use ordinary frequentist estimates: n_i / n_tot
        self.prob_y = calc_log_prob(data_y)
        f = partial(calc_log_prob, alpha=self.alpha)
        self.prob_xi_y = [
            dict(train_data_x[col].groupby(train_data_y).agg(f)) 
            for col in train_data_x.columns
        ]

    def predict(self, data_x: pd.DataFrame):
        # Here you need to, for each sample in data_x, calculate the most probable class, and return them all in a list
        # That is, combine the estimates above using the naive Bayesian formula.
        # You don't necessarily need to normalize the probabilities, since you will only pick the most probable class.
        probas = {}
        for class_ in self.prob_y:
            p = pd.Series(np.full(len(data_x), self.prob_y[class_]))
            for prob_xi_y, col in zip(self.prob_xi_y, data_x.columns):
                p += data_x[col].map(prob_xi_y[class_])
            probas[class_] = p

        probas_df = pd.DataFrame(probas)
        best_idx = probas_df.values.argmax(axis=1)
        return pd.Series(probas_df.columns[best_idx])


In [8]:
# When you are done, check the performance
mod = MyNaiveBayes()
mod.fit(train_data_x, train_data_y)

In [9]:
# Performance on training data
sum(mod.predict(train_data_x) == train_data_y)

1086

In [10]:
# Performance on test data
sum(mod.predict(test_data_x) == test_data_y)

987

In [11]:
# Maybe you experienced some problems with your first implementation?
# For example, did you just multiply all factors together in 'predict'?
# What happens when you multiply very many very small numbers?
# If you had this problem, change your implementation to add the logarithms of the probabilities
# instead of multiplying them. Did it help?

Surprisingly I did not have any issues with the multiplication of very small 
numbers. In theory, this should cause some issues with the accuracy since the 
numbers get smaller and smaller and you lose some precision as you multiple 
regularly sized integers with a very small one. I guess the current data is not
large enough for me to get this issue.

In [12]:
# Possibly you also got some errors or warnings related to 'division by zero', for the test data.
# Why do you think this happens?
# But don't worry. We deal with that next.

I've been used to adding a lot of epsilon values in a lot of machine learning
related code, so this time I also added an epsilon value to accommodate the 
zeros

In [13]:
# Now it is time to change your code to use Bayesian estimates of the probabilities. 
# Consider using a parameter alpha to control the strength of the prior: (n_i + alpha/2) / (n_tot + alpha)
# Redefine your code up there, or copy it here, as you prefer. Then evaluate the model again below

In [14]:
# Check the performance of the new model
alpha = 1.0
mod = MyNaiveBayes(alpha)
mod.fit(train_data_x, train_data_y)

In [15]:
# Performance on training data
sum(mod.predict(train_data_x) == train_data_y)

1055

In [16]:
# Performance on test data
sum(mod.predict(test_data_x) == test_data_y)

936

In [17]:
# How did the different versions perform?
# You can also test with some different alpha.
# What conclusions can be drawn?
# What have you learned?

In [18]:
for alpha in range(0, 5):
    mod = MyNaiveBayes(alpha)
    mod.fit(train_data_x, train_data_y)

    print(f"Alpha = {alpha}")
    print(sum(mod.predict(train_data_x) == train_data_y))
    print(sum(mod.predict(test_data_x) == test_data_y))

Alpha = 0
1086
987
Alpha = 1
1055
936
Alpha = 2
972
867
Alpha = 3
918
833
Alpha = 4
888
812


The version with higher alpha values performed worst when looking at the 
performance, however, formula wise, using the one with the alpha does not 
have an issue with 0 probabilities by default.

Conclusions are:

* Naive Bayes algorithms can perform well on specific problems, even with
  some of the assumptions it has with the data.
* An alpha parameter can provide some regularization/smoothing effect on 
  model. It performed worse on the given dataset, but there are probably 
  datasets where the a higher value performs better.

Learnings:

* Implementation of the Naive Bayes algorithm. This includes specifics as 
  such the importance of log probabilities and alpha parameter to avoid zero
  division.
* Impact of different alphas on the given dataset. In this case, lower 
  alpha is better.